In [2]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import IMDB
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader

ModuleNotFoundError: No module named 'torch'

In [ ]:
# Define the LSTM model
class SentimentLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, dropout):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embedding_dim, sparse=True)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        lstm_out, _ = self.lstm(embedded)
        lstm_out = lstm_out[:, -1, :]  # Take the last output of LSTM
        output = self.fc(lstm_out)
        return output

In [ ]:
# Define the LSTM model
class SentimentLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, dropout):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embedding_dim, sparse=True)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        lstm_out, _ = self.lstm(embedded)
        lstm_out = lstm_out[:, -1, :]  # Take the last output of LSTM
        output = self.fc(lstm_out)
        return output

In [1]:
# Function to train the model
def train_model(model, train_loader, optimizer, criterion, epochs):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for text, offsets, labels in train_loader:
            optimizer.zero_grad()
            output = model(text, offsets)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader)}')

# Function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for text, offsets, labels in test_loader:
            output = model(text, offsets)
            predicted = torch.argmax(output, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    accuracy = correct / total
    print(f'Test Accuracy: {accuracy:.4f}')

# Load and prepare the data
def load_data(data_directory, split):
    tokenizer = get_tokenizer("basic_english")
    train_data, test_data = IMDB(tokenizer=tokenizer)
    train_data, valid_data = to_map_style_dataset(train_data)
    vocab = build_vocab_from_iterator(train_data['text'], specials=["<unk>"])
    vocab.set_default_index(vocab["<unk>"])
    return train_data, valid_data

# Main function to train and evaluate the model
def main(data_directory="aclImdb/train", split=0.8, batch_size=64, embedding_dim=100, hidden_dim=256,
         num_layers=2, dropout=0.5, epochs=5):
    # Load data
    train_data, test_data = load_data(data_directory, split)
    train_loader, test_loader = prepare_data_loader(train_data, test_data, batch_size)

    # Initialize the model
    vocab_size = len(train_data['text'].get_vocab())
    output_dim = 2  # 2 classes: positive and negative
    model = SentimentLSTM(vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, dropout)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    # Train the model
    train_model(model, train_loader, optimizer, criterion, epochs)

    # Evaluate the model
    evaluate_model(model, test_loader)

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'torch'